In [ ]:
!pip install ultralytics
!pip install opencv-python
!pip install matplotlib
!pip install tqdm

import os
import yaml
import random
import torch
import numpy as np
from IPython.display import display, Image
from ultralytics import YOLO
from pathlib import Path
import shutil
from tqdm.notebook import tqdm

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    device = torch.device("cpu")
    print("Running on CPU")

yaml_content = """
path: /fab3/btech/2022/vijay.singh22b/Final Object Data  
train: images/train
val: images/val
test: images/test
names:
  0: animal
  1: vehicle
  2: person
  3: speedbump
  4: obstacle
  5: sign
  6: greenlight
  7: redlight
  8: orangelight
nc: 9
task: detect
"""

yaml_path = "./Final Object Data/data.yaml"

os.makedirs(os.path.dirname(yaml_path), exist_ok=True)

with open(yaml_path, "w") as f:
    f.write(yaml_content)

print(f"Custom YAML file created at {yaml_path}")

def display_yaml(yaml_path):
    with open(yaml_path, "r") as f:
        content = f.read()
    print("YAML Configuration:")
    print(content)

display_yaml(yaml_path)

import datetime
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
project_dir = f"yolo_training_{timestamp}"
run_name = "training_run"
checkpoint_dir = os.path.join(project_dir, run_name, "weights")

def train_yolo_model(yaml_path, resume=False, epochs=50, batch_size=16, img_size=640):
    print(f"Loading YAML configuration from: {yaml_path}")
    
    last_checkpoint = os.path.join(checkpoint_dir, "last.pt")
    resume_path = None
    
    if resume and os.path.exists(last_checkpoint):
        print(f"Resuming training from checkpoint: {last_checkpoint}")
        resume_path = last_checkpoint
        model = YOLO(resume_path)
    else:
        model = YOLO('yolov8x.pt')
    
    results = model.train(
        data=yaml_path,
        epochs=50,
        batch=batch_size,
        imgsz=img_size,
        patience=15,
        device=0,
        workers=8,
        pretrained=True,
        optimizer="SGD",
        lr0=0.01,
        lrf=0.001,
        momentum=0.937,
        weight_decay=0.0005,
        warmup_epochs=3.0,
        warmup_momentum=0.8,
        warmup_bias_lr=0.1,
        box=7.5,
        cls=0.5,
        dfl=1.5,
        augment=True,
        val=True,
        save=True,
        save_period=5,
        project=project_dir,
        name=run_name,
        exist_ok=True,
        resume=resume_path
    )
    
    print(f"Model training completed. Checkpoints saved in {checkpoint_dir}")
    return model, results

def verify_dataset_structure(base_path):
    required_dirs = [
        "images/train",
        "images/val",
        "labels/train",
        "labels/val"
    ]
    
    missing = []
    for d in required_dirs:
        full_path = os.path.join(base_path, d)
        if not os.path.exists(full_path):
            missing.append(full_path)
            
    if missing:
        print("WARNING: The following directories are missing:")
        for m in missing:
            print(f"  - {m}")
        print("\nYou need to create these directories and organize your data accordingly.")
    else:
        print("Dataset structure verified successfully!")
        
    return len(missing) == 0

def run_training(resume=False):
    with open(yaml_path, 'r') as f:
        config = yaml.safe_load(f)
    
    base_path = config.get('path', './dataset')
    print(f"Dataset base path: {base_path}")
    
    if not verify_dataset_structure(base_path):
        print("Please organize your dataset according to the YAML configuration before training.")
        return
    
    print("Starting model training...")
    model, results = train_yolo_model(
        yaml_path=yaml_path,
        resume=resume,
        epochs=100,
        batch_size=16,
        img_size=640
    )
    
    print(f"Training completed. Model saved at {checkpoint_dir}")
    return model

import glob

def main():
    if os.path.exists(checkpoint_dir) and len(glob.glob(os.path.join(checkpoint_dir, "*.pt"))) > 0:
        print("Found existing training session. Attempting to resume...")
        model = run_training(resume=True)
    else:
        print("Starting new training session...")
        model = run_training(resume=False)
    
    print("Training process completed!")

main()